# Sentiment Analysis with Deep Learning using Mocdified BERT

## Exploratory Data Analysis

In [ ]:
#libraries and packages
!pip install torch
!pip install ranger-adabelief==0.1.0 
!pip install transformers

import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from nltk import word_tokenize, pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.wsd import lesk

from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
import nltk
import string
import time
import jsonpickle


In [ ]:
# Consume:
CONSUMER_KEY = 'SS97NM79TEdKPsSWlmnhUcOwd'
CONSUMER_SECRET = 'rZpFgeEMsCp6cYZ4u2gSTVj9sFXQUMVh5lVkDAEV2b3MTEGyE2'

# Access:
ACCESS_TOKEN = '1037289423321350144-aG0QZqLs0cOHIuwNgagYIRTZVT40b4'
ACCESS_SECRET = 'O4qSyIY74M7iUqQB2Qgb92H8AwIvCV5O3SjSA8rJFbc10'


def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api


# Create API object
api = connect_to_twitter_OAuth()

In [ ]:
"""# Extracting the tweets"""

def get_save_tweets(filepath, api, query,max_tweets=1000, lang='en'):
    tweetCount = 0

    # Open file and save tweets
    with open(filepath, 'w') as f:
        # Send the query
        for tweet in tweepy.Cursor(api.search_tweets, q=query, lang=lang, since_id='2017-01-01').items(max_tweets):
        # for tweet in tweepy.Cursor(api.search_tweets, q=query, lang=lang).items(max_tweets):
            # Convert to JSON format
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

        # Display how many tweets we have collected
        print("Downloaded {0} tweets".format(tweetCount))
query = 'airlines'# change the query topic Ex:'BuzzFeed' for Business tweets 
get_save_tweets('Airlines.json', api, query)# change the filename ".json" for different dataset

In [ ]:
def tweets_to_df(path):
    tweets = list(open(path, 'rt'))

    text = []
    weekday = []
    month = []
    day = []
    hour = []
    hashtag = []
    url = []
    favorite = []
    reply = []
    retweet = []
    follower = []
    following = []
    user = []
    screen_name = []

    for t in tweets:
        t = jsonpickle.decode(t)

        # Text
        text.append(t['text'])

        # Decompose date
        date = t['created_at']
        weekday.append(date.split(' ')[0])
        month.append(date.split(' ')[1])
        day.append(date.split(' ')[2])

        time = date.split(' ')[3].split(':')
        hour.append(time[0])

        # Has hashtag
        if len(t['entities']['hashtags']) == 0:
            hashtag.append(0)
        else:
            hashtag.append(1)

        # Has url
        if len(t['entities']['urls']) == 0:
            url.append(0)
        else:
            url.append(1)

        # Number of favs
        favorite.append(t['favorite_count'])

        # Is reply?
        if t['in_reply_to_status_id'] == None:
            reply.append(0)
        else:
            reply.append(1)

            # Retweets count
        retweet.append(t['retweet_count'])

        # Followers number
        follower.append(t['user']['followers_count'])

        # Following number
        following.append(t['user']['friends_count'])

        # Add user
        user.append(t['user']['name'])

        # Add screen name
        screen_name.append(t['user']['screen_name'])

    d = {'TweetText': text,
         'weekday': weekday,
         'month': month,
         'day': day,
         'hour': hour,
         'has_hashtag': hashtag,
         'has_url': url,
         'fav_count': favorite,
         'is_reply': reply,
         'retweet_count': retweet,
         'followers': follower,
         'following': following,
         'user': user,
         'screen_name': screen_name
         }

    return pd.DataFrame(data=d)
df =  tweets_to_df('Airlines.json')

In [ ]:
df.to_csv("AirlinesTweets.csv",sep = ",", index=False)

In [ ]:
#load data
df = pd.read_csv('AirlinesTweets.csv')


In [ ]:
#preview
df.head()

In [ ]:
#info
df.info()

In [ ]:
#check for null
df.isnull().sum()

In [ ]:
#look at an example
df.TweetText.iloc[10]

In [ ]:
#count for each class
df.day.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# #plot class distribution
# plt.figure(figsize=(10, 5))
# sns.countplot(df.lsentiment, palette='Spectral')
# plt.xlabel('Classes')
# plt.title('Class Distribution');

In [ ]:
#store classes into an array
possible_labels = df.day.unique()
possible_labels

In [ ]:
#convert labels into numeric values
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
#convert labels into numeric values
df['day'] = df.day.replace(label_dict)
df.head(10)


In [ ]:
#need equal length sentences
#plot hist of sentence length
plt.figure(figsize=(10, 5))
sns.histplot([len(s) for s in df.TweetText], bins=100)
plt.title('Sentence Length')
plt.show()

In [ ]:
#find the maximum length
max_len = max([len(sent) for sent in df.TweetText])
print('Max length: ', max_len)

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

#train test split
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                   df.day.values,
                                                   test_size = 0.2,
                                                   random_state = 17,
                                                   stratify = df.day.values)

In [ ]:
#create new column
df['data_type'] = ['not_set'] * df.shape[0]
df.head()

In [ ]:
#fill in data type
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['day', 'data_type']).count()

## Tokenization

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
#load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

In [ ]:
#tokenize train set
encoded_data_train = tokenizer.batch_encode_plus(df[df.data_type == 'train'].TweetText.values,
                                                add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

In [ ]:
#tokenizer val set
encoded_data_val = tokenizer.batch_encode_plus(df[df.data_type == 'val'].TweetText.values,
                                                #add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

In [ ]:
encoded_data_train

## Encoding

In [ ]:
#encode train set
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].day.values)

In [ ]:
#encode val set
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']

#convert data type to torch.tensor
labels_val = torch.tensor(df[df.data_type == 'val'].day.values)

In [ ]:
input_ids_train

In [ ]:
attention_masks_train

In [ ]:
labels_train

In [ ]:
#create dataloader
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                             attention_masks_val, 
                             labels_val)

In [ ]:
print(len(dataset_train))
print(len(dataset_val))

In [ ]:
dataset_train

In [ ]:
dataset_train.tensors

## Set Up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

#load pre-trained BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = len(label_dict),
                                                      output_attentions = False,
                                                      output_hidden_states = False)

In [ ]:
#model summary
model.config

## Create Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4 #since we have limited resource

#load train set
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = batch_size)

#load val set
dataloader_val = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = 32) #since we don't have to do backpropagation for this step

In [ ]:
from transformers import Adafactor,get_linear_schedule_with_warmup
epochs = 10
#load optimizer
from ranger_adabelief import RangerAdaBelief
optimizer = RangerAdaBelief(model.parameters(), lr=1e-3, eps=1e-12, betas=(0.9,0.999))

In [ ]:
#load scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = len(dataloader_train)*epochs)

## Define Performance Metrics

In [ ]:
import numpy as np
from sklearn import metrics

# #f1 score
# def f1_score_func(preds, labels):
#     preds_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()
#     return f1_score(labels_flat, preds_flat, average = 'weighted')
def eval_metrics(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = metrics.accuracy_score(labels_flat, preds_flat)
    prec = metrics.precision_score(labels_flat, preds_flat, average = 'weighted')
    recall = metrics.recall_score(labels_flat, preds_flat, average = 'weighted')
    f1score = metrics.f1_score(labels_flat, preds_flat, average = 'weighted')
    conf_mat= metrics.confusion_matrix(labels_flat, preds_flat)
    return acc,prec,recall,f1score,conf_mat

In [ ]:
#accuracy score
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
def evaluate(dataloader_val):

    #evaluation mode disables the dropout layer 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        #compute accuracy
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

## Train Model

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
# for epoch in range(epoch_num):

    #set model in train mode
    model.train()

    #tracking variable
    loss_train_total = 0
    
    #set up progress bar
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        #set gradient to 0
        model.zero_grad()

        #load into GPU
        batch = tuple(b.to(device) for b in batch)

        #define inputs
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)
        loss = outputs[0] #output.loss
        loss_train_total +=loss.item()

        #backward pass to get gradients
        # loss.backward()
        
        #clip the norm of the gradients to 1.0 to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        #update optimizer
        optimizer.step()

        #update scheduler
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    tqdm.write('\nEpoch {epoch}')
    
    #print training result
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    #evaluate
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    #f1 score
    Acc,Precision,Recall,F1_Score,conf_mat = eval_metrics(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {Acc}')
    tqdm.write(f'Precision (weighted): {Precision}')
    tqdm.write(f'Recall (weighted): {Recall}')
    tqdm.write(f'F1 Score (weighted): {F1_Score}')

## Model Evaluation


In [ ]:
outputs.loss

In [ ]:
outputs.logits

In [ ]:
#save model
model.to(device)
pass

In [ ]:
#evaluate
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
#get accuracy score
accuracy_per_class(predictions, true_vals)

In [ ]:
Acc,Precision,Recall,F1_Score,conf_matrix = eval_metrics(predictions, true_vals)

In [ ]:
print("Accuracy:",Acc, "Precision:",Precision,"Recall :",Recall,"F1_Score:",F1_Score)

In [ ]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
print("confusion_matrix",conf_matrix)

In [ ]:
model.parameters